In [1]:
import os
import sys
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [2]:
# Helper function
def newPath(path):
    if not os.path.isdir(path):
        os.mkdir(path)
        
def writeProgress(msg, count, total):
    sys.stdout.write(msg + "{:.2%}\r".format(count/total))
    sys.stdout.flush()

In [3]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Load numpy array

In [4]:
all_npy = np.load('./npy/all_2372.npy')
movie_genre = np.load('./npy/movie_genre.npy')
usr_following = np.load('./npy/user_followings.npy')
usr_genre = np.load('./npy/user_genre.npy')

print('All features:', all_npy.shape)
print('Movie genre:', movie_genre.shape)
print('User following:', usr_following.shape)
print('User genre:', usr_genre.shape)

All features: (165, 2372)
Movie genre: (165, 20)
User following: (1582, 165)
User genre: (1582, 20)


In [5]:
usr_nb = len(usr_following) # the number of users
movie_nb = len(movie_genre)  # the number of movies
print(usr_nb, movie_nb)

usr_test_amount = 150
movie_test_amount = 32
print(usr_test_amount, movie_test_amount)

latent_dim = 64 # latent dims
ft_dim = all_npy.shape[1] # feature dims
print(latent_dim, ft_dim)

1582 165
150 32
64 2372


### Normalize usr_genre

In [6]:
usr_genre_norm = np.zeros(usr_genre.shape)
for i in range(len(usr_genre)):
    usr_genre_norm[i] = usr_genre[i]/np.max(usr_genre[i])
print(usr_genre_norm.shape)
# print('Before:', usr_genre)
# print('After:', usr_genre_norm)

(1582, 20)


# Training & testing split
## Prepare
### User

In [7]:
#The number of following movie for each user
each_user = np.sum(usr_following, axis=1)
# print(each_user)

print('Min number of followings:', np.min(each_user))
print('Max number of followings:', np.max(each_user))
print('Avg of followers:', np.mean(each_user))

asc = np.sort(each_user)
# print(each_user)
# print(asc)
desc = np.flip(asc)
# print(desc)

Min number of followings: 10
Max number of followings: 133
Avg of followers: 14.820480404551201


In [8]:
print('== 10:', np.sum(each_user == 10))
equal10_idx = np.nonzero(each_user == 10)[0]
print(equal10_idx.shape, equal10_idx)

== 10: 394
(394,) [   0    2    5   14   16   18   24   28   33   36   42   44   46   48
   50   58   60   66   72   75   86   90   98  106  107  114  115  116
  120  125  129  132  137  138  148  150  153  154  155  159  161  162
  166  170  176  178  179  182  185  187  189  192  198  200  203  206
  207  213  215  217  227  236  237  239  252  258  264  267  269  270
  274  275  280  282  284  287  289  291  297  301  308  316  321  322
  324  338  339  343  345  348  349  351  363  366  370  373  379  392
  403  405  407  409  413  422  426  435  436  437  441  456  463  465
  470  483  484  485  491  492  495  503  504  506  511  512  514  519
  523  526  527  531  532  533  538  542  544  552  554  555  572  576
  580  582  585  593  595  596  597  603  604  609  611  620  621  626
  630  634  636  637  639  643  644  648  652  655  658  660  663  665
  671  677  682  705  715  717  721  727  733  735  745  751  752  753
  755  761  762  764  767  778  781  787  809  810  816  81

In [9]:
random.seed(42)
test_idx = sorted(random.sample(list(equal10_idx), 100))
print(len(test_idx), test_idx)

100 [14, 46, 48, 58, 60, 98, 106, 120, 138, 150, 153, 161, 176, 182, 187, 192, 198, 213, 239, 275, 301, 316, 322, 343, 403, 409, 437, 441, 463, 465, 470, 484, 491, 492, 503, 519, 552, 554, 555, 572, 580, 585, 604, 611, 648, 663, 715, 721, 733, 753, 761, 764, 767, 787, 819, 827, 875, 914, 923, 966, 975, 992, 994, 1060, 1115, 1118, 1134, 1142, 1151, 1160, 1191, 1207, 1209, 1232, 1243, 1248, 1269, 1274, 1277, 1282, 1295, 1313, 1341, 1375, 1377, 1400, 1431, 1441, 1453, 1455, 1476, 1487, 1492, 1494, 1499, 1506, 1533, 1536, 1546, 1565]


In [10]:
st = set()
for idx in test_idx:
    print('Index:', idx)
    print('Sum:', usr_following[idx].sum())
    print(usr_following[idx])
    li = list(np.where(usr_following[idx] == 1)[0])
    print(li)
    st = st | set(li)
    print(len(st), st)
    print('==================================================')

Index: 14
Sum: 10
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[6, 50, 70, 85, 88, 95, 98, 107, 136, 150]
10 {98, 70, 6, 136, 107, 50, 85, 150, 88, 95}
Index: 46
Sum: 10
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[57, 67, 77, 93, 105, 136, 141, 147, 149, 158]
19 {98, 67, 70, 6, 136, 105, 107, 77, 141, 50, 147, 85, 150, 149, 88, 57, 93, 158, 95}
Index: 48
Sum: 10
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[18, 52, 57, 60, 80, 118, 126, 127, 129, 134]
145 {0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 62, 64, 66, 67, 68, 70, 71, 72, 74, 75, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164}
Index: 1248
Sum: 10
[0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [11]:
usr_test_amount = len(test_idx)
movie_test_amount = 32 #len(st)
print(usr_test_amount, movie_test_amount)

100 32


## Setup 

In [12]:
# init
random.seed(42)
train_t = []
train_f = []
test_t = []
test_f = []

for i in range(usr_nb):
    # init
    t_for_train = []
    f_for_train = []
    t_for_test = []
    f_for_test = []
    
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(movie_nb):
            if usr_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
                
        train_t.append(t_for_train)
        train_f.append(f_for_train)
#         print(len(t_for_train) + len(f_for_train))
        
    else: #if in test id, choose half of true and other 
        temp_t = []
        temp_f = []
        
        for j in range(movie_nb):
            if usr_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        
        # random choose half true and half false for test 
        t_for_test = random.sample(temp_t, math.ceil(0.5*len(temp_t)))
        f_for_test = random.sample(temp_f, movie_test_amount-len(t_for_test))
        
        test_t.append(t_for_test)
        test_f.append(f_for_test)
        
        #the others for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t.append(t_for_train)
        train_f.append(f_for_train)
        
    if not (len(t_for_train) + len(f_for_train) + len(t_for_test) + len(f_for_test)) == movie_nb:
        print('Error!!!')
        break

In [13]:
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_f))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1582
The length of train_f: 1582
The length of test_t: 100
The length of test_f: 100


## Stat

In [14]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train / usr_nb
print('Training:', avg)

#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test / usr_test_amount
print('Testing:', avg)

Training: 14.504424778761061
Testing: 5.0


In [15]:
all_auxilary = [i for i in range(movie_nb)]

# Recommendation model

# Testing Part

# Metrics

## Top N

In [16]:
def F1_score(prec,rec):
    f1 = 2*((prec*rec)/(prec+rec))
    return f1

def topN(RSls, n):
    maxn = np.argsort(RSls)[::-1][:n]
    return maxn

## NDCG

In [17]:
def allSortPrepare(testRS):
    all_sort = []

    for i in range(usr_test_amount):
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))

    all_sort = np.asarray(all_sort)
    print(all_sort.shape)
    return all_sort

In [18]:
def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg = 0
    for i in range(len(prec_list)):
        dcg += (2**prec_list[i]-1)/math.log2(i+2)
    return dcg

def NDCG(target, testRS, num_ndcg, all_sort): #target是真正的喜好
    total_ndcg = 0
    
    for m in range(usr_test_amount): # the number of testing users
        idcg = DCG(target[m][:num_ndcg])
        
        pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        total_ndcg += ndcg
        
    avg_ndcg = total_ndcg/usr_test_amount
    return avg_ndcg

## MAP

In [19]:
from sklearn.metrics import average_precision_score

def MAP(target,testRS):
    total_prec = 0
    for u in range(usr_test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec += average_precision_score(y_true, y_scores)
        
    Map_value = total_prec/usr_test_amount
    
    return Map_value

In [20]:
def metrics(testRS, target, sumtarget, all_sort):
    print('\n==============================\n')
    # Top N
    N = [1, 5]
    correct = 0

    for n in N:
        print('Top', n)
        correct = 0

        for i in range(len(testRS)):
            topn = topN(testRS[i], n)
            sum_target = int(np.sum(target[i]))

            TP = 0
            for i in topn:
                if i < sum_target:
                    TP += 1

            correct += TP

        prec = correct/(len(testRS)*n) #150*n
        recall = correct/sumtarget

        print('prec:', prec)
        print('recall:', recall)
        print('F1_score:', F1_score(prec, recall))
        print('*****')

    print('\n==============================\n')

    # NDCG
    num_ndcgs = [10]
    for num_ndcg in num_ndcgs:
        print('NDCG@', num_ndcg)
        print('NDCG score:', NDCG(target, testRS, num_ndcg, all_sort))
        print('*****')

    print('\n==============================\n')

    # MAP
    print('MAP:', MAP(target,testRS))
    print('\n==============================\n')

In [21]:
'''
usr_test_amount = 147
movie_test_amount = 32
'''
def testing(U, Y, A, E, Au, Ay, Aa, Av, B):
    #with Embedding
    result = np.zeros((usr_test_amount, movie_nb))
    RS = np.zeros((usr_test_amount, movie_nb))

    #test_idx --> Test 的 index length = 150
    sum_alpha = 0
    test_yes_id = []

    for s in range(usr_test_amount):
#         print(s, test_idx[s])

        yes = []
        sample = train_t[test_idx[s]]
        alpha = np.zeros([len(sample)])

        for a in range(len(sample)):
            r = np.max(movie_genre[sample[a]] * usr_genre_norm[test_idx[s]]) #sample a 的category vec *user_category vec

    # #         ''' Observe each part in attention
    #         WuUu = np.sum(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T))
    #         WyYy = np.sum(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T))
    #         WaAa = np.sum(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T))
    #         WvVy = np.sum(np.dot(np.dot(Av[test_idx[s]], E),np.expand_dims(all_npy[sample[a]],0).T))
    #         print('The sum of each par -->',
    #               '\nw1:',testW1,
    #               '\nWuU:',WuUu,
    #               '\nwyY:',WyYy,
    #               '\nWaA:',WaAa,
    #               '\nWvV:',WvVy)
    # #         '''

            alpha_a = (np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T) + 
                       np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T) + 
                       np.dot(Aa[test_idx[s]][sample[a]],np.expand_dims(A[sample[a]],0).T) +
                       np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_npy[sample[a]],0).T)))


            # relu part
            alpha[a]=np.sum((relu(alpha_a)))*r
            # tanh part
    #         alpha[a]=np.sum((np.tanh(alpha_a)))*r

        mul = np.zeros((1,latent_dim))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        sum_alpha += np.sum(alpha)

#         print("{:<15}{}".format('sum_alpha:', sum_alpha))
#         print('==================================================')

        for i in range(len(sample)):
            mul += norm_alpha[i] * A[sample[i]] # attention alpha*Ai part
        new_mul = mul + U[test_idx[s]]  #(U+auxilary)

        for k in range(movie_nb):
            result[s][k] = np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T) + np.dot(B[test_idx[s]], np.dot(E, all_npy[k].T))
        
    #取出test的資料
    print(RS.shape)

    testRS = np.zeros((usr_test_amount, movie_test_amount)) #shape 150 * 32
    target = np.zeros((usr_test_amount, movie_test_amount)) #shape 150 * 32

    for z in range(usr_test_amount):
        user_id = test_idx[z]
        # positive target YouTuber list
        youtube_t = test_t[z] 
        # not target YouTuber list
        youtube_f = test_f[z]

    #     print(user_id)
    #     print(youtube_t)
    #     print(youtube_f)

        #前面放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = 1

        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

    #     print(testRS[z])
    #     print(target[z])
    #     print('==============================')

    print(target.shape, testRS.shape)
    sumtarget = np.sum(target)
    print('num of positive data in testing:', sumtarget) # whole matrix: 4800

    # for metrics
    metrics(testRS, target, sumtarget, allSortPrepare(testRS))

# Evaluate weight

In [22]:
npzs = ['MRM_E240_11.npz']
npzs

['MRM_E240_11.npz']

# Get latent factor and Each weight

In [23]:
for SAVE_NAME in npzs:
    print(SAVE_NAME)
    
    params = np.load('./weight/grid/' + SAVE_NAME)
#     print(params)
    U = params['U']
    Y = params['Y']
    A = params['A']
    E = params['E']
    Au = params['Wu']
    Ay = params['Wy']
    Aa = params['Wa']
    Av = params['Wv']
    B = params['B']

#     print('User latent shape: ',U.shape)
#     print('photo latent shape: ', Y.shape)
#     print('Auxilary latent shape: ',A.shape)
#     print('Embedding shape:', E.shape)
#     print('Wu weight shape:', Au.shape)
#     print('Wy weight shape:', Ay.shape)
#     print('Wa weight shape:', Aa.shape)
#     print('Wv weight shape:', Av.shape)
#     print('Beta shape:',B.shape)

    testing(U, Y, A, E, Au, Ay, Aa, Av, B)
    print('==================================================')

MRM_E240_11.npz
(100, 165)
(100, 32) (100, 32)
num of positive data in testing: 500.0
(100, 32)


Top 1
prec: 0.4
recall: 0.08
F1_score: 0.13333333333333333
*****
Top 5
prec: 0.29
recall: 0.29
F1_score: 0.29
*****


NDCG@ 10
NDCG score: 0.4462832648614355
*****


MAP: 0.3909743086328884


